In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False

# DF experiments

In [ ]:
exp_name = "2021_04_30_stepper";  # old buzzer
#exp_name = "2021_02_23_wall";  # old buzzer
#exp_name = "2021_02_25_wall"
# new buzzer

mic_type = "audio_deck"
#motors = "all45000"
motors = 0

source = "sweep_new"

## spectrogram analysis

In [ ]:
# get first distance measurement from the chosen datatype. 
all_data = pd.read_pickle(f'../experiments/{exp_name}/all_data.pkl')
row_idx = 1
data = all_data.loc[(all_data.mic_type==mic_type) & (all_data.motors==motors)].iloc[row_idx]
data

In [ ]:
from frequency_analysis import get_spectrogram_raw
print(data.seconds)
spec, freqs = get_spectrogram_raw(data.frequencies_matrix, data.stft)
spec = np.abs(spec)
print(spec.shape, freqs.shape)

In [ ]:
from plotting_tools import pcolorfast_custom
fig, ax = plt.subplots()
pcolorfast_custom(ax, data.seconds, freqs, spec[:, 0, :])
ax.set_xlabel('time')
ax.set_ylabel('frequency')

## raw df analysis

In [ ]:
from wall_detector import WallDetector

wall_detector = WallDetector(exp_name, mic_type)
backup_exists = wall_detector.fill_from_backup(
    exp_name, mic_type, motors, appendix="_raw"
)
wall_detector.remove_nan_rows(verbose=True)

In [ ]:
wall_detector.df
for distance, df_chosen in wall_detector.df.groupby('distance'):
    plt.figure()
    for i_mic, df_mic in df_chosen.groupby('mic'):
        i_mic = int(i_mic)
        df_mic = df_mic.sort_values(by='time')
        label=f'mic{i_mic}, N={len(df_mic)}'
        plt.scatter(df_mic.time, df_mic.frequency.values, color=f'C{int(i_mic)}', 
                    label=label)
    plt.title(f'distance {distance}')
    plt.xlabel('time')
    plt.ylabel('peak frequency')
    plt.legend(loc='upper left')
    break

In [ ]:
import seaborn as sns

def plot_current_df():
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5)
    sns.scatterplot(
        data=wall_detector.df, x="frequency", y="magnitude", hue="distance", linewidth=0
    )
    ax.set_yscale('log')
    min_y = max(wall_detector.df.magnitude.min(), 1e-2)
    max_y = min(wall_detector.df.magnitude.max(), 200)
    ax.set_ylim(min_y, max_y)

In [ ]:
plot_current_df()

In [ ]:
wall_detector.remove_bad_measurements(verbose=True)
plot_current_df()

In [ ]:
wall_detector.remove_bad_freqs(verbose=True)
plot_current_df()

In [ ]:
wall_detector.merge_close_freqs(verbose=True)
plot_current_df()

In [ ]:
wall_detector.remove_outliers(verbose=True)
plot_current_df()

In [ ]:
wall_detector.remove_spurious_freqs(verbose=True)
plot_current_df()

In [ ]:
wall_detector.remove_nan_rows(verbose=True)

In [ ]:
from plotting_tools import plot_df_matrix
from wall_detector import prune_df_matrix

matrix, dists, freqs = wall_detector.get_df_matrix()
matrix_pruned, freqs_pruned, *_ = prune_df_matrix(matrix, freqs, ratio_missing_allowed=0.5)

fig, axs = plt.subplots(1, matrix.shape[0])
fig.set_size_inches(5 * matrix.shape[0], 5)
fig_pruned, axs_pruned = plt.subplots(1, matrix.shape[0])
fig_pruned.set_size_inches(5 * matrix.shape[0], 5)
for i, (ax, ax_pruned) in enumerate(zip(axs, axs_pruned)):
    plot_df_matrix(dists, freqs, matrix[i], ax=ax)
    ax.set_ylabel('raw \n frequency [Hz]')
    plot_df_matrix(dists, freqs_pruned, matrix_pruned[i], ax=ax_pruned)
    ax_pruned.set_ylabel('pruned \n frequency [Hz]')
    ax.set_title(f"mic{i}")

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('./../crazyflie-audio/python/'))
from bin_selection import generate_sweep, get_frequencies

bins, *_ =  generate_sweep(source)
freqs_buzzer = get_frequencies()[bins]

if source == "sweep_new":
    freqs = [2999, 3113, 3236, 3370, 3514, 3632, 3757, 3891, 3986, 4137, 4245, 4358, 4477, 4603, 4737, 4878]
    print('error between played and frequency bin:', freqs_buzzer - freqs)

plt.figure()
[plt.plot([f, f], [1, 2], color='C0') for f in freqs_buzzer]
[plt.plot([f, f], [2, 3], color='C1') for f in freqs_pruned]
[plt.plot([f, f], [3, 4], color='C2') for f in freqs]
plt.plot([], [], color='C0', label='buzzer') 
plt.plot([], [], color='C1', label='pruned')
plt.plot([], [], color='C2', label='original')
plt.legend()
pass